In [1]:
import re
import pandas as pd
from tqdm import tqdm

In [2]:
def parse_zdna_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    zdna_list = []
    current_scaffold = None

    for line in lines:
        line = line.strip()
        if line.startswith('NW_') or line.startswith('GCF_'):
            current_scaffold = line
        elif line and current_scaffold:
            try:
                start, end = map(int, line.split())
                zdna_list.append((current_scaffold, (start, end)))
            except ValueError:
                continue  # skip lines that do not have valid start and end positions
    
    return zdna_list

In [3]:
def parse_gff(file_path):
    # Read the GFF file into a pandas DataFrame
    gff_df = pd.read_csv(file_path, sep='\t', comment='#', header=None,
                         names=['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'])
    return gff_df

In [4]:
def categorize_zdna(zdna_list, gff_df):
    zdna_counts = {
        'exon': 0,
        'intron': 0,
        'promoter': 0,
        'downstream': 0,
        'intergenic': 0
    }

    for scaffold, (start, end) in tqdm(zdna_list, total=len(zdna_list)):
        # Check if ZDNA is in exon
        exons = gff_df[(gff_df['seqid'] == scaffold) & (gff_df['type'] == 'exon') & (gff_df['strand'] == '+')]
        if any((exons['start'] <= start) & (exons['end'] >= end)):
            zdna_counts['exon'] += 1
            continue

        # Check if ZDNA is in intron
        genes = gff_df[(gff_df['seqid'] == scaffold) & (gff_df['type'] == 'gene') & (gff_df['strand'] == '+')]
        if any((genes['start'] <= start) & (genes['end'] >= end)):
            zdna_counts['intron'] += 1
            continue
        
        # Check if ZDNA is in promoter region
        if any((genes['start'] - 1000 <= start) & (genes['start'] >= start)):
            zdna_counts['promoter'] += 1
            continue

        # Check if ZDNA is in downstream region
        if any((genes['end'] <= end) & (genes['end'] + 200 >= end)):
            zdna_counts['downstream'] += 1
            continue

        # If not in any other category, it's intergenic
        zdna_counts['intergenic'] += 1
    
    return zdna_counts

In [5]:
zdna_file_path = 'text_predictions.txt'
gff_file_path  = '../ncbi_dataset/data/GCF_000787575.1/genomic.gff'

In [6]:
gff_df = parse_gff(gff_file_path)

In [7]:
gff_df[(gff_df['seqid'] == "NW_012236532.1") & (gff_df['type'] == 'exon') & (gff_df['strand'] == '+')]

,seqid,source,type,start,end,score,strand,phase,attributes
91938,NW_012236532.1,RefSeq,exon,10542,11073,.,+,.,ID=exon-XM_012904211.1-1;Parent=rna-XM_0129042...
91939,NW_012236532.1,RefSeq,exon,11149,17518,.,+,.,ID=exon-XM_012904211.1-2;Parent=rna-XM_0129042...
91944,NW_012236532.1,RefSeq,exon,17798,18544,.,+,.,ID=exon-XM_012904212.1-1;Parent=rna-XM_0129042...
91945,NW_012236532.1,RefSeq,exon,18620,18665,.,+,.,ID=exon-XM_012904212.1-2;Parent=rna-XM_0129042...
91954,NW_012236532.1,RefSeq,exon,20411,20472,.,+,.,ID=exon-XM_012904214.1-1;Parent=rna-XM_0129042...
...,...,...,...,...,...,...,...,...,...
96143,NW_012236532.1,RefSeq,exon,1330710,1330847,.,+,.,ID=exon-XM_012904771.1-2;Parent=rna-XM_0129047...
96144,NW_012236532.1,RefSeq,exon,1330933,1332114,.,+,.,ID=exon-XM_012904771.1-3;Parent=rna-XM_0129047...
96145,NW_012236532.1,RefSeq,exon,1332541,1332918,.,+,.,ID=exon-XM_012904771.1-4;Parent=rna-XM_0129047...
96146,NW_012236532.1,RefSeq,exon,1332994,1337180,.,+,.,ID=exon-XM_012904771.1-5;Parent=rna-XM_0129047...


In [8]:
zdna_list = parse_zdna_file(zdna_file_path)

In [9]:
zdna_counts = categorize_zdna(zdna_list, gff_df)
print(zdna_counts)

100%|██████████| 5799/5799 [06:49<00:00, 14.15it/s]

{'exon': 2487, 'intron': 102, 'promoter': 224, 'downstream': 94, 'intergenic': 2892}


In [10]:
len(zdna_list)

5799

### Count using zhunt data

In [24]:
zhunt=pd.read_csv("zhunt.bed", sep="\t", names=['Scaffold', 'Start', 'End', 'Score'])
len(zhunt)

4328

In [25]:
zhunt

,Scaffold,Start,End,Score
0,NW_012236428.1,731,761,1237.247
1,NW_012236428.1,1359,1382,1096.390
2,NW_012236431.1,1627,1664,1440.850
3,NW_012236432.1,44,72,2539.351
4,NW_012236435.1,2125,2149,1030.985
...,...,...,...,...
4323,NW_012236532.1,1257074,1257094,1016.263
4324,NW_012236532.1,1258304,1258328,2099.467
4325,NW_012236532.1,1282423,1282443,1041.981
4326,NW_012236532.1,1284649,1284669,1232.502


In [26]:
# convert to list of tuples
zhunt_zdna_regions = [(scaffold, (start, end)) for scaffold, start, end, _ in zhunt.values]
zdna_counts = categorize_zdna(zhunt_zdna_regions, gff_df)

100%|██████████| 4328/4328 [05:17<00:00, 13.65it/s]


In [27]:
print(zdna_counts)

{'exon': 1196, 'intron': 153, 'promoter': 773, 'downstream': 115, 'intergenic': 2091}


In [19]:
# count quadruplexes
import re
pattern="(?:G{3,}[ATGC]{1,7}){3,}G{3,}"
pattern_C="(?:C{3,}[ATGC]{1,7}){3,}C{3,}"

In [ ]:
# open fasta file
with open("../ncbi_dataset/data/GCF_000787575.1/GCF_000787575.1_Asub_2.0_genomic.fna") as file:
    sequence = file.read()

# remove lines starting with '>'
sequence = ''.join(sequence.split('\n')[1:])


In [20]:
PQS=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern,sequence,re.IGNORECASE)] #re.IGNORECASE отвечает за игнорирования отличий между заглавными и строчными буквами
len(PQS) #выводим число найденных квадруплексов

NameError: name 'sequence' is not defined

In [21]:
PQS_minus=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern_C,sequence,re.IGNORECASE)] #re.IGNORECASE отвечает за игнорирования отличий между заглавными и строчными буквами
len(PQS_minus) #выводим число найденных квадруплексов

NameError: name 'sequence' is not defined

In [ ]:
# categorize and count quadruplexes on both strands
def categorize_pqs(PQS, PQS_minus, gff_df) -> dict:
    pqs_counts = {
        'exon': 0,
        'intron': 0,
        'promoter': 0,
        'downstream': 0,
        'intergenic': 0
    }

    # count PQS on the plus strand
    for start, end, _ in PQS:
        categorized = False

        # Check if PQS is in exon on plus strand
        exons = gff_df[(gff_df['type'] == 'exon') & (gff_df['strand'] == '+')]
        if any((exons['start'] <= start) & (exons['end'] >= end)):
            pqs_counts['exon'] += 1
            categorized = True
            
        # Check if PQS is in intron on plus strand
        genes = gff_df[(gff_df['type'] == 'gene') & (gff_df['strand'] == '+')]
        if any((genes['start'] <= start) & (genes['end'] >= end)):
            pqs_counts['intron'] += 1
            categorized = True
            
        # Check if PQS is in promoter region on plus strand
        if any((genes['start'] - 1000 <= start) & (genes['start'] >= start)):
            pqs_counts['promoter'] += 1
            categorized = True
            
        # Check if PQS is in downstream region on plus strand
        if any((genes['end'] <= end) & (genes['end'] + 200 >= end)):
            pqs_counts['downstream'] += 1
            categorized = True
            
        # If not in any other category, it's intergenic
        if not categorized:
            pqs_counts['intergenic'] += 1
            
    # count PQS on the minus strand
    for start, end, _ in PQS_minus:
        categorized = False

        # Check if PQS is in exon on minus strand
        exons = gff_df[(gff_df['type'] == 'exon') & (gff_df['strand'] == '-')]
        if any((exons['start'] <= start) & (exons['end'] >= end)):
            pqs_counts['exon'] += 1
            categorized = True
            
        # Check if PQS is in intron on minus strand
        genes = gff_df[(gff_df['type'] == 'gene') & (gff_df['strand'] == '-')]
        if any((genes['start'] <= start) & (genes['end'] >= end)):
            pqs_counts['intron'] += 1
            categorized = True
            
        # Check if PQS is in promoter region on minus strand
        if any((genes['start'] - 1000 <= start) & (genes['start'] >= start)):
            pqs_counts['promoter'] += 1
            categorized = True
            
        # Check if PQS is in downstream region on minus strand
        if any((genes['end'] <= end) & (genes['end'] + 200 >= end)):
            pqs_counts['downstream'] += 1
            categorized = True
            
        # If not in any other category, it's intergenic
        if not categorized:
            pqs_counts['intergenic'] += 1
            
    return pqs_counts